In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

### Data

In [2]:
df = pd.read_csv('data_20180119_20180420.csv', index_col=0)
df.head()

,t0,t1,K1,pi1,t2,K2,pi2
0,2018-01-02,2018-01-19,120.0,13.525,2018-04-20,120.0,61.050
1,2018-01-02,2018-01-19,110.0,92.525,2018-04-20,118.0,71.275
2,2018-01-02,2018-01-19,100.0,191.525,2018-04-20,112.0,108.400
3,2018-01-02,2018-01-19,117.5,27.125,2018-04-20,116.0,82.500
4,2018-01-02,2018-01-19,117.0,30.450,2018-04-20,150.0,3.675


In [3]:
t0List = list(df.t0.unique())

# dimension: K[k-th t0][t1 and t2][i-th option with i-th strike]
K = [[np.array(df.loc[df.t0 == t0,'K1']),np.array(df.loc[df.t0==t0,'K2'])] for t0 in t0List]

# dimension: Pi[k-th t0][t1 and t2][i-th option with i-th price]
Pi = [[np.array(df.loc[df.t0 == t0,'pi1']),np.array(df.loc[df.t0==t0,'pi2'])] for t0 in t0List]

In [4]:
# number of t0
N_t0 = len(t0List)

# number of times (t1, t2)
T = 2

# number of options in hedging
N1, N2 = 20, 20
Size = max(N1, N2)

In [5]:
# Assume marginal distribution is uniform
def margDistr(batch_size):
    data = np.random.uniform(size=(batch_size, T))
    for t in range(T):
        data[:, t] = S_min + (S_max - S_min) * data[:, t] # just a simple test, need further consideration
    return data

In [6]:
# arbitrary
S_t0 = 143
S_max = 180
S_min = 100
BATCH_SIZE = 4 # for ease

S = margDistr(BATCH_SIZE)

In [7]:
Phi = (S[:,1]-S[:,0])
# define the payoff function of the derivative

### Variables

In [8]:
# parameter: d
d = tf.Variable(
    initial_value = tf.random.normal(shape=[N_t0, 1], dtype=tf.float64),
    trainable = True,
    name = 'd'
)

In [9]:
# parameter: lambda
Lambda = tf.Variable(
    initial_value = tf.random.normal(shape=[N_t0, T, Size], dtype=tf.float64),
    trainable = True,
    name = 'lambda'
)

In [10]:
# parameter: Delta
Delta = tf.Variable(
    initial_value = tf.random.normal(shape=[N_t0, T-1], dtype=tf.float64),
    trainable = True,
    name = 'Delta'
)

Question: 
$\Delta_1(S_1)$ is actually a function of $S_1$, should we treat the whole term as the variable or assume its formula is something like $\Delta_1(S_1) = \delta_1S_1$ and treat $\delta_1$ as the variable we need to optimise?

### The step-by-step computation of the objective function

$d+\sum_{i=1}^{N_1}\lambda_{i,1}\Pi_{i,1}+\sum_{i=1}^{N_2}\lambda_{i,2}\Pi_{i,2}$

In [11]:
hedgePrice = []
for n in range(N_t0):
    p = d[n]
    for t in range(T):
        p += tf.reduce_sum(Lambda[n,t,:]*Pi[n][t][:])
    hedgePrice.append(p)

In [12]:
# take a look
hedgePrice

[<tf.Tensor: shape=(1,), dtype=float64, numpy=array([890.64463977])>,
 <tf.Tensor: shape=(1,), dtype=float64, numpy=array([277.64992917])>,
 <tf.Tensor: shape=(1,), dtype=float64, numpy=array([401.84222244])>,
 <tf.Tensor: shape=(1,), dtype=float64, numpy=array([-622.03040404])>,
 <tf.Tensor: shape=(1,), dtype=float64, numpy=array([-215.90788441])>,
 <tf.Tensor: shape=(1,), dtype=float64, numpy=array([-912.94438297])>,
 <tf.Tensor: shape=(1,), dtype=float64, numpy=array([806.74059786])>,
 <tf.Tensor: shape=(1,), dtype=float64, numpy=array([871.94900119])>,
 <tf.Tensor: shape=(1,), dtype=float64, numpy=array([283.79872458])>,
 <tf.Tensor: shape=(1,), dtype=float64, numpy=array([-935.51027626])>,
 <tf.Tensor: shape=(1,), dtype=float64, numpy=array([-503.46666635])>,
 <tf.Tensor: shape=(1,), dtype=float64, numpy=array([-229.43346824])>,
 <tf.Tensor: shape=(1,), dtype=float64, numpy=array([-497.44295917])>]

$d+\sum_{i=1}^{N_1}\lambda_{i,1}(S_1-K_{i,1})^++\sum_{i=1}^{N_2}\lambda_{i,2}(S_2-K_{i,2})^++\Delta_{1}(S_1)\cdot(S_2-S_1)$

In [13]:
hedgeTerm = []
for n in range(N_t0):
    p = d[n]
    for t in range(T):
        p += tf.reduce_sum(Lambda[n,t,:]*np.maximum(S[:,t].reshape(BATCH_SIZE,1)-K[n][t][:],0),axis=1)
    for t in range(T-1):
        p += Delta[n,t]*(S[:,t+1]-S[:,t])
    hedgeTerm.append(p)

In [14]:
# take a look
hedgeTerm

[<tf.Tensor: shape=(4,), dtype=float64, numpy=array([ 61.12567126, 563.39181668, 392.70992908, 549.88238569])>,
 <tf.Tensor: shape=(4,), dtype=float64, numpy=array([ 9.98207725, -7.42418731, 13.31652611, 27.42385417])>,
 <tf.Tensor: shape=(4,), dtype=float64, numpy=array([ -3.99731906, 302.36409496, 238.72902249, 142.73155783])>,
 <tf.Tensor: shape=(4,), dtype=float64, numpy=array([ -22.128626  , -257.04617554, -199.78338327, -136.97338241])>,
 <tf.Tensor: shape=(4,), dtype=float64, numpy=array([  -3.37584791, -132.55240528,  -85.14710927, -147.97942427])>,
 <tf.Tensor: shape=(4,), dtype=float64, numpy=array([ -18.30671979, -342.815695  , -257.49022841, -253.46084188])>,
 <tf.Tensor: shape=(4,), dtype=float64, numpy=array([ 24.90805493, 372.6060004 , 255.51315787, 370.08137135])>,
 <tf.Tensor: shape=(4,), dtype=float64, numpy=array([ 22.11722831, 133.96709698, 121.34791993, 165.89985885])>,
 <tf.Tensor: shape=(4,), dtype=float64, numpy=array([  0.47020306, 106.56846919,  79.36107018,  

$\inf_{h\in\mathcal{H}^m}\int hd\mu_0+\int \beta_\gamma(c-h)d\theta$

$d+\sum_{i=1}^{N_1}\lambda_{i,1}\Pi_{i,1}+\sum_{i=1}^{N_2}\lambda_{i,2}\Pi_{i,2} + \Gamma\cdot\left[\left(\Phi(S_1,S_2)-d-\sum_{i=1}^{N_1}\lambda_{i,1}(S_1-K_{i,1})^+-\sum_{i=1}^{N_2}\lambda_{i,2}(S_2-K_{i,2})^+-\Delta_{1}(S_1)\cdot(S_2-S_1)\right)^+\right]^2$

In [15]:
# objective function
Price = tf.reduce_mean(hedgePrice, axis=1)

Gamma = 100
diff = Phi - tf.stack(hedgeTerm, axis=0)
Penalty = tf.reduce_mean(tf.square(tf.nn.relu(diff)), axis=1)

ObjFunc = Price + Gamma*Penalty

In [16]:
# take a look
Price

<tf.Tensor: shape=(13,), dtype=float64, numpy=
array([ 890.64463977,  277.64992917,  401.84222244, -622.03040404,
       -215.90788441, -912.94438297,  806.74059786,  871.94900119,
        283.79872458, -935.51027626, -503.46666635, -229.43346824,
       -497.44295917])>

In [17]:
# take a look
Penalty

<tf.Tensor: shape=(13,), dtype=float64, numpy=
array([0.00000000e+00, 6.71164304e+02, 5.03725794e+01, 4.24266390e+04,
       2.03314691e+04, 7.86120898e+04, 0.00000000e+00, 0.00000000e+00,
       2.56010303e+01, 9.21620482e+04, 1.12366541e+04, 5.28041552e+04,
       6.22570246e+04])>

In [18]:
# take a look
ObjFunc

<tf.Tensor: shape=(13,), dtype=float64, numpy=
array([8.90644640e+02, 6.73940803e+04, 5.43910016e+03, 4.24204187e+06,
       2.03293100e+06, 7.86029604e+06, 8.06740598e+02, 8.71949001e+02,
       2.84390175e+03, 9.21526931e+06, 1.12316194e+06, 5.28018609e+06,
       6.22520502e+06])>

### Optimisation

In [19]:
# Combine all the computation as a function
def ObjFunc(d, Lambda, Delta, K, Pi, Phi, S):
    hedgePrice = []
    for n in range(N_t0):
        p = d[n]
        for t in range(T):
            p += tf.reduce_sum(Lambda[n,t,:]*Pi[n][t][:])
        hedgePrice.append(p)
    
    hedgeTerm = []
    for n in range(N_t0):
        p = d[n]
        for t in range(T):
            p += tf.reduce_sum(Lambda[n,t,:]*np.maximum(S[:,t].reshape(BATCH_SIZE,1)-K[n][t][:],0),axis=1)
        for t in range(T-1):
            p += Delta[n,t]*(S[:,t+1]-S[:,t])
        hedgeTerm.append(p)
    
    Price = tf.reduce_mean(hedgePrice, axis=1)
    Gamma = 100
    diff = Phi-tf.stack(hedgeTerm, axis=0)
    Penalty = tf.reduce_mean(tf.square(tf.nn.relu(diff)), axis=1)
    
    return Price + Gamma*Penalty, Price

In [20]:
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate = 0.01)

In [24]:
for step in range(10000):
    with tf.GradientTape() as tape:
        obj, price = ObjFunc(d, Lambda, Delta, K, Pi, Phi, S)
    gradients = tape.gradient(obj, [d, Lambda, Delta])
    optimizer.apply_gradients(zip(gradients, [d, Lambda, Delta]))

In [25]:
price

<tf.Tensor: shape=(13,), dtype=float64, numpy=
array([-12232.3879989 ,  -4972.46447127, -12594.35459494,  -1133.03988455,
        -2742.17076983,   -984.28663393, -14638.46317849, -11480.31678387,
        -9666.07969813,   -598.60105114,  -1850.94043755,   -305.06549677,
         -959.68033254])>

In [26]:
obj

<tf.Tensor: shape=(13,), dtype=float64, numpy=
array([-12232.3879989 ,  -4972.46447127, -12594.35459494,  -1132.65256285,
        -2741.83241418,   -984.19764283, -14638.46317849, -11475.55201419,
        -9665.96378062,   -598.55462667,  -1850.94043755,   -305.04014335,
         -959.65181002])>